# Remove before submtion for our refrenc only
## Yusuf update 
I completed the code for part one, but there's a huge error in the calculation of the inverse kinematics. I increased the max number of function evaluations to 5000 to give the optimizer more chances to find a solution, but the errors are still quite large.

## The Three Transformation Matrices

T_base:
- Transforms from the "world" coordinate system to our robot's base coordinate system.
- Accounts for where our robot is positioned/oriented in the world.

T_final:
- The transformation we calculate using D‑H parameters.
- Goes from the robot's base frame to the end‑effector frame.
- This is the forward kinematics computes we did in lab 2 part 4.

T_tool:
- Transforms from the end‑effector frame to the actual tool frame.
- Accounts for any gripper attached to the end‑effector.

Overall Transformation:
- The overall transformation from the world frame to the tool frame is given by:
    $$T_{\text{total}} = T_{\text{base}} \; T_{\text{final}} \; T_{\text{tool}}$$
- Since our world frame is the same as the robot base frame then T_base is the identity matrix. 
- So, if we have a tool attached, the overall transformation simplifies to:
  $$T_{\text{total}} = T_{\text{final}} \; T_{\text{tool}}$$
- If no tool is attached, T_tool is also the identity matrix, and $$T_{\text{total}} = T_{\text{final}}$$


In [1]:
import math
import numpy as np
import pandas as pd
from sympy.matrices import Matrix
from scipy.optimize import least_squares
from sympy  import symbols, cos, sin, atan2, pi, lambdify 
from pathlib import Path

In [2]:
#Global variables
q1, q2, q3, q4, q5, q6 = symbols('q1 q2 q3 q4 q5 q6') # Define symbolic variables for joint angles
right = pi/2
L1, L2, L3 = symbols('L1 L2 L3') # Define symbolic variables for link lengths

- Recursively searches the current working directory (Path.cwd()) for files named `mecharm_control_group_5.csv`.
- Selects the first match whose parent folder path contains the substring `Lab3`.
- If no suitable file is found it raises a FileNotFoundError with instructions on where to place the CSV or how to change the notebook working directory.
- If found, it prints the file path and loads the CSV into a pandas DataFrame named `data`.
- `known_joint_angles` is a NumPy array extracted from the DataFrame containing joint angles J1 to J6.
- `known_position` is a NumPy array extracted from the DataFrame containing end-effector positions X, Y, Z.
- `known_orientation` is a NumPy array extracted from the DataFrame containing end-effector orientations RX, RY, RZ.

In [3]:
for p in Path.cwd().rglob('mecharm_control_group_5.csv'):
    if 'Lab3' in str(p.parent):
        csv_path = p
        break

if csv_path.exists() == False:
    raise FileNotFoundError("mecharm_control_group_5.csv not found. Put it in Lab3/ under the repo or set notebook cwd to the repo root (use %cd).")
else:
    print("Loading CSV from:", csv_path)
    data = pd.read_csv(csv_path)
    
#Joint Angles from Lab 2 task 3
known_joint_angles = data[['J1','J2','J3','J4','J5','J6']].to_numpy()
#End-Effector Positions from Lab 2 task 3
known_position = data[['X','Y','Z']].to_numpy()
#End-Effector Orientations from Lab 2 task 3
known_orientation = data[['RX','RY','RZ']].to_numpy()

Loading CSV from: /Users/yusufdineh/Documents/GitHub/EE347/Lab3/mecharm_control_group_5.csv


- we are defining our DH table parameters here in a 6x4 matrix called `dh_table`
- DH table rows: [a, alpha, d, offset, offset]

In [ ]:
dh_table = [
    [0,         -right,       114,  q1],           # joint 1
    [100,           0,         0,   q2 - right],   # joint 2
    [10,        -right,       0,   q3],           # joint 3
    [0,         right,      96,   q4],           # joint 4
    [0,         -right,       0,   q5],           # joint 5
    [0,         0,         56.5,   q6]              # joint 6
]


# dh_table = [
#     [0,      0,         131.56,  q1],           # joint 1
#     [0,      right,     0,       q2 - right],   # joint 2  
#     [-110.4, 0,         0,       q3],           # joint 3
#     [-96,    0,         64.62,   q4 - right],   # joint 4
#     [0,      right,     73.18,   q5 + right],   # joint 5
#     [0,      -right,    48.6,    q6]            # joint 6
# ]

- `get_transformation_matrix(a, alpha, d, theta)`
  - Inputs: the four DH parameters for one joint:
  - Returns: a 4×4 SymPy homogeneous transform Matrix that encodes the rotation and translation from frame i to frame i+1 for that DH row.

- `overall_transformation(dh_table)`
  - `T = Matrix(np.identity(4))` initializes the cumulative transform as the 4×4 identity (base frame).  
  - Loop: for each DH row `a, alpha, d, theta` it calls `get_transformation_matrix` to get the per‑joint transform `T_i`, then multiplies `T = T * T_i`. That sequential multiplication composes transforms frame‑by‑frame, so after the loop `T` is the full symbolic transform from the robot base to the end‑effector.  
  - Return: the composed 4×4 SymPy Matrix (use `T[:3,3]` for position and `T[:3,:3]` for rotation when lambdifying).

In [5]:
def get_transformation_matrix(a, alpha, d, theta):
    return Matrix([
        [cos(theta), -sin(theta), 0, a],
        [sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -sin(alpha)*d],
        [sin(theta)*sin(alpha), cos(theta)*sin(alpha), cos(alpha), cos(alpha)*d],
        [0, 0, 0, 1]
    ])
def overall_transformation(dh_table):
    T = Matrix(np.identity(4))
    for i in range(len(dh_table)):
        a, alpha, d, theta = dh_table[i]
        T_i = get_transformation_matrix(a, alpha, d, theta)
        T = T * T_i
    return T

- `T_symbolic` Builds a single 4×4 SymPy homogeneous transform from base → end‑effector using your DH table. Result is symbolic (contains q1..q6).
- `fk_num` Create a fast numeric function that maps numeric (q1..q6) → position vector (x,y,z).
- `rot_num` = Create a fast numeric function that maps numeric (q1..q6) → 3×3 rotation matrix.

In [6]:
T_symbolic = overall_transformation(dh_table)
fk_num = lambdify((q1, q2, q3, q4, q5, q6), T_symbolic[:3, 3], modules='numpy')
rot_num = lambdify((q1, q2, q3, q4, q5, q6), T_symbolic[:3, :3], modules='numpy')

`fk` computes end‑effector position [x,y,z] from joint angles using the lambdified forward‑kinematics function fk_num and returns it as a NumPy array.

In [7]:
def fk(q_vals):
    q = np.asarray(q_vals, dtype=float)
    q = np.radians(q)
    xyz = np.asarray(fk_num(*q), dtype=float).ravel()
    return xyz


# Inverse kinematics helper functions

- `position_error`: compute the error between the forward-kinematics position and the desired target.
- `orientation_error`: compute rotational error (ZYX yaw-pitch-roll) between current forward-kinematics rotation and desired orientation.
  - Helper `_wrap_angle(a)` wraps any angle to [-π, π].
- `link_lengths` computea a upper bound on robot reach from DH rows.

In [8]:
def rotation_matrix_to_euler_zyx(R, transpose=False):
    """
    Extract Z-Y-X Euler angles (yaw, pitch, roll) from a 3x3 rotation matrix.
    
    The matrix convention is:
    R = [[cos(γ)cos(β),  -cos(α)sin(γ) + cos(γ)sin(α)sin(β),   sin(γ)sin(α) + cos(γ)cos(α)sin(β)],
         [cos(β)sin(γ),   cos(γ)cos(α) + sin(γ)sin(α)sin(β),  -cos(γ)sin(α) + cos(α)sin(γ)sin(β)],
         [-sin(β),        cos(β)sin(α),                         cos(α)cos(β)]]
    
    Where: γ = yaw, β = pitch, α = roll
    
    Args:
        R: 3x3 rotation matrix
        transpose: If True, use R.T instead of R (default False)
        
    Returns:
        (roll, pitch, yaw) in radians
    """
    # Apply transpose if needed
    if transpose:
        R = R.T
    
    # Extract pitch from R[2,0] = -sin(β)
    sin_beta = -R[2, 0]
    sin_beta = np.clip(sin_beta, -1.0, 1.0)  # Clamp for numerical stability
    pitch = np.arcsin(sin_beta)
    
    cos_beta = np.cos(pitch)
    
    # Check for gimbal lock (cos(β) ≈ 0, i.e., pitch ≈ ±90°)
    if np.abs(cos_beta) < 1e-6:
        # Gimbal lock: set yaw = 0 and extract roll from remaining matrix elements
        yaw = 0.0
        roll = np.arctan2(-R[0, 1], R[1, 1])
    else:
        # Normal case: extract roll and yaw from matrix elements
        # From R[2,1] = cos(β)sin(α)  and  R[2,2] = cos(α)cos(β)
        roll = np.arctan2(R[2, 1] / cos_beta, R[2, 2] / cos_beta)
        
        # From R[0,0] = cos(γ)cos(β)  and  R[1,0] = cos(β)sin(γ)
        yaw = np.arctan2(R[1, 0] / cos_beta, R[0, 0] / cos_beta)
    
    return roll, pitch, yaw

In [9]:
def position_error(q_position, x_target, y_target, z_target):
    """
    Compute position error between FK result and target.
    
    Args:
        q_position: Joint angles in degrees (6-element array)
        x_target, y_target, z_target: Target position in mm
        
    Returns:
        3-element error vector in mm
    """
    pos = fk(q_position)  # fk() handles degree→radian conversion internally
    return np.array([pos[0] - float(x_target),
                     pos[1] - float(y_target),
                     pos[2] - float(z_target)])

def orientation_error(q_orientation, rx_d, ry_d, rz_d):
    """
    Compute orientation error (roll, pitch, yaw) between FK result and target.
    
    Args:
        q_orientation: Joint angles in degrees (6-element array)
        rx_d, ry_d, rz_d: Target orientation in degrees (roll, pitch, yaw)
        
    Returns:
        3-element error vector in radians (for optimizer consistency)
    """
    def _wrap_angle(a):
        return (a + np.pi) % (2*np.pi) - np.pi

    q = np.asarray(q_orientation, dtype=float)  # Input in degrees
    q_rad = np.radians(q)  # Convert to radians for FK computation
    rx_d_rad = np.radians(rx_d)
    ry_d_rad = np.radians(ry_d)
    rz_d_rad = np.radians(rz_d)

    R = np.array(rot_num(*q_rad), dtype=float)   # 3x3 rotation matrix
    roll, pitch, yaw = rotation_matrix_to_euler_zyx(R)

    # All computations now in radians
    err_roll  = _wrap_angle(roll  - rx_d_rad)
    err_pitch = _wrap_angle(pitch - ry_d_rad)
    err_yaw   = _wrap_angle(yaw   - rz_d_rad)

    return np.array([err_roll, err_pitch, err_yaw])

def link_lengths(dh_table):
    """Compute upper bound on robot reach from DH parameters."""
    link_lengths = 0.0
    for a, alpha, d, theta in dh_table:
        link_lengths += abs(float(a)) + abs(float(d))
    return link_lengths

- `inverse_kinematics` computes joint angles [q1..q6] to reach desired end-effector pose using numerical optimization.

In [10]:
def inverse_kinematics(x_target, y_target, z_target, rx_d, ry_d, rz_d, q_init, link_lengths, max_iterations=5000, tolerance=1e-6):
    """
    Compute joint angles to reach desired end-effector pose.
    
    All angles in degrees (input/output).
    Positions in mm, orientations in degrees.
    
    Args:
        x_target, y_target, z_target: Target position in mm
        rx_d, ry_d, rz_d: Target orientation in degrees
        q_init: Initial joint angle guess in degrees (6-element array)
        link_lengths: Maximum reach of robot in mm
        max_iterations: Max function evaluations for optimizer
        tolerance: Convergence tolerance
        
    Returns:
        q_sol: Joint angles in degrees (6-element array)
    """
    distance_from_origin = math.hypot(x_target, y_target, z_target)
    if distance_from_origin > link_lengths:
        raise ValueError("Target outside of range")
    
    q_init = np.asarray(q_init, dtype=float)
    
    def combined_residual(q):
        """
        Combined position AND orientation error.
        All 6 joints optimized simultaneously.
        q is in degrees throughout.
        """
        pos_err = position_error(q, x_target, y_target, z_target)
        ori_err = orientation_error(q, rx_d, ry_d, rz_d)
        return np.concatenate((pos_err, ori_err))
    
    # Solve all 6 joints simultaneously
    result = least_squares(combined_residual, q_init, max_nfev=max_iterations, xtol=tolerance)
    
    if result.success or result.cost < 1e-3:
        return result.x  # Returns in degrees
    else:
        raise ValueError(f"Inverse kinematics did not converge (cost: {result.cost:.6f})")

## Validation and Accuracy Check

## Transformation Matrix Validation

Test the validity of transformation matrices by checking mathematical properties and DH parameter implementation.

In [11]:
def test_transformation_matrices():
    """
    Test the validity of transformation matrices by checking:
    1. Individual DH transformation matrices are valid
    2. Overall transformation matrix properties
    3. Rotation matrix orthogonality
    4. Determinant properties
    """
    print("Testing Transformation Matrix Validity")
    print("=" * 50)
    
    # Test 1: Check individual DH transformation matrices
    print("\n1. Testing Individual DH Transformation Matrices:")
    print("-" * 45)
    
    test_joints = [0, 45, -30, 90, -45, 60]  # Test joint angles in degrees
    test_joints_rad = np.radians(test_joints)
    
    for i, (a, alpha, d, theta_sym) in enumerate(dh_table):
        print(f"\nJoint {i+1}: a={a}, α={float(alpha)*180/np.pi:.1f}°, d={d}, θ=θ{i+1}")
        
        # Substitute test angle into symbolic theta
        theta_test = theta_sym.subs([(q1, test_joints_rad[0]), (q2, test_joints_rad[1]), 
                                    (q3, test_joints_rad[2]), (q4, test_joints_rad[3]), 
                                    (q5, test_joints_rad[4]), (q6, test_joints_rad[5])])
        
        # Get numeric transformation matrix
        T_i = get_transformation_matrix(a, alpha, d, theta_test)
        T_i_num = np.array(T_i.evalf(), dtype=float)
        
        # Extract rotation part (3x3)
        R_i = T_i_num[:3, :3]
        
        # Check if rotation matrix is orthogonal: R @ R.T should be identity
        should_be_identity = R_i @ R_i.T
        orthogonality_error = np.linalg.norm(should_be_identity - np.eye(3))
        
        # Check determinant should be +1 for proper rotation
        det_R = np.linalg.det(R_i)
        
        print(f"  Rotation orthogonality error: {orthogonality_error:.2e}")
        print(f"  Rotation determinant: {det_R:.6f} (should be 1.0)")
        print(f"  Bottom row: {T_i_num[3, :]} (should be [0, 0, 0, 1])")
        
        # Check if this is a valid transformation matrix
        is_valid = (orthogonality_error < 1e-10) and (abs(det_R - 1.0) < 1e-10) and \
                   np.allclose(T_i_num[3, :], [0, 0, 0, 1])
        
        print(f"  ✅ Valid transformation matrix" if is_valid else f"  ❌ Invalid transformation matrix")
    
    # Test 2: Check overall transformation matrix
    print(f"\n\n2. Testing Overall Transformation Matrix:")
    print("-" * 42)
    
    # Compute overall transformation for test joint angles
    T_total = overall_transformation(dh_table)
    T_total_subs = T_total.subs([(q1, test_joints_rad[0]), (q2, test_joints_rad[1]), 
                                (q3, test_joints_rad[2]), (q4, test_joints_rad[3]), 
                                (q5, test_joints_rad[4]), (q6, test_joints_rad[5])])
    T_total_num = np.array(T_total_subs.evalf(), dtype=float)
    
    # Extract rotation and position
    R_total = T_total_num[:3, :3]
    p_total = T_total_num[:3, 3]
    
    # Check overall rotation matrix properties
    orthogonality_error_total = np.linalg.norm(R_total @ R_total.T - np.eye(3))
    det_R_total = np.linalg.det(R_total)
    
    print(f"Overall transformation for joint angles: {test_joints}°")
    print(f"End-effector position: [{p_total[0]:.2f}, {p_total[1]:.2f}, {p_total[2]:.2f}] mm")
    print(f"Rotation orthogonality error: {orthogonality_error_total:.2e}")
    print(f"Rotation determinant: {det_R_total:.6f} (should be 1.0)")
    print(f"Bottom row: {T_total_num[3, :]} (should be [0, 0, 0, 1])")
    
    is_valid_overall = (orthogonality_error_total < 1e-10) and (abs(det_R_total - 1.0) < 1e-10) and \
                       np.allclose(T_total_num[3, :], [0, 0, 0, 1])
    
    print(f"✅ Valid overall transformation matrix" if is_valid_overall else f"❌ Invalid overall transformation matrix")
    
    # Test 3: Check consistency between symbolic and numeric functions
    print(f"\n\n3. Testing Symbolic vs Numeric Function Consistency:")
    print("-" * 52)
    
    # Use the lambdified functions
    pos_lambdified = fk(test_joints)  # This uses fk() which converts degrees to radians
    R_lambdified = np.array(rot_num(*test_joints_rad), dtype=float)
    
    # Compare with direct symbolic evaluation
    pos_symbolic = T_total_num[:3, 3]
    R_symbolic = T_total_num[:3, :3]
    
    pos_diff = np.linalg.norm(pos_lambdified - pos_symbolic)
    rot_diff = np.linalg.norm(R_lambdified - R_symbolic)
    
    print(f"Position difference (lambdified vs symbolic): {pos_diff:.2e} mm")
    print(f"Rotation difference (lambdified vs symbolic): {rot_diff:.2e}")
    
    consistency_ok = (pos_diff < 1e-10) and (rot_diff < 1e-10)
    print(f"✅ Functions are consistent" if consistency_ok else f"❌ Functions are inconsistent")
    
    # Test 4: Check for singularities at zero configuration
    print(f"\n\n4. Testing Zero Configuration:")
    print("-" * 32)
    
    zero_joints = [0, 0, 0, 0, 0, 0]
    pos_zero = fk(zero_joints)
    q_rad_zero = np.radians(zero_joints)
    R_zero = np.array(rot_num(*q_rad_zero), dtype=float)
    
    print(f"Zero configuration position: [{pos_zero[0]:.2f}, {pos_zero[1]:.2f}, {pos_zero[2]:.2f}] mm")
    print(f"Zero configuration rotation matrix:")
    print(R_zero)
    
    det_zero = np.linalg.det(R_zero)
    ortho_zero = np.linalg.norm(R_zero @ R_zero.T - np.eye(3))
    
    print(f"Determinant at zero: {det_zero:.6f}")
    print(f"Orthogonality error at zero: {ortho_zero:.2e}")
    
    # Test 5: Quick DH parameter sanity check
    print(f"\n\n5. DH Parameter Sanity Check:")
    print("-" * 33)
    
    print("DH Table:")
    print("Joint |   a   | alpha |   d   |  theta")
    print("------|-------|-------|-------|--------")
    for i, (a, alpha, d, theta) in enumerate(dh_table):
        alpha_deg = float(alpha) * 180 / np.pi
        print(f"  {i+1}   | {a:5.1f} | {alpha_deg:5.1f}° | {d:5.1f} | θ{i+1}")
    
    # Calculate theoretical maximum reach
    max_reach_theoretical = sum(abs(float(a)) + abs(float(d)) for a, alpha, d, theta in dh_table)
    print(f"\nTheoretical maximum reach: {max_reach_theoretical:.1f} mm")
    
    # Test extreme positions to see if they're reasonable
    extreme_positions = [
        [90, 0, 0, 0, 0, 0],    # Joint 1 rotated
        [0, 90, 0, 0, 0, 0],   # Joint 2 rotated  
        [0, 0, 90, 0, 0, 0],   # Joint 3 rotated
    ]
    
    print(f"\nTesting extreme positions:")
    for i, joints in enumerate(extreme_positions):
        pos = fk(joints)
        distance = np.linalg.norm(pos)
        print(f"  Config {joints}: pos=[{pos[0]:.1f}, {pos[1]:.1f}, {pos[2]:.1f}], distance={distance:.1f} mm")
    
    print(f"\n" + "=" * 50)
    print("TRANSFORMATION MATRIX TEST COMPLETE")
    print("=" * 50)

# Run the test
test_transformation_matrices()

Testing Transformation Matrix Validity

1. Testing Individual DH Transformation Matrices:
---------------------------------------------

Joint 1: a=4, α=-90.0°, d=0, θ=θ1
  Rotation orthogonality error: 0.00e+00
  Rotation determinant: 1.000000 (should be 1.0)
  Bottom row: [0. 0. 0. 1.] (should be [0, 0, 0, 1])
  ✅ Valid transformation matrix

Joint 2: a=5, α=0.0°, d=0, θ=θ2
  Rotation orthogonality error: 1.43e-17
  Rotation determinant: 1.000000 (should be 1.0)
  Bottom row: [0. 0. 0. 1.] (should be [0, 0, 0, 1])
  ✅ Valid transformation matrix


2. Testing Overall Transformation Matrix:
------------------------------------------
Overall transformation for joint angles: [0, 45, -30, 90, -45, 60]°
End-effector position: [9.00, 0.00, 0.00] mm
Rotation orthogonality error: 1.43e-17
Rotation determinant: 1.000000 (should be 1.0)
Bottom row: [0. 0. 0. 1.] (should be [0, 0, 0, 1])
✅ Valid overall transformation matrix


3. Testing Symbolic vs Numeric Function Consistency:
----------------

## Forward Kinematics Validation

Test the accuracy of our forward kinematics by comparing computed positions and orientations with known values from the CSV data.

In [12]:
def test_forward_kinematics():
    """
    Test forward kinematics accuracy by comparing computed positions/orientations 
    with known values from the CSV data.
    """
    print("Testing Forward Kinematics Accuracy")
    print("=" * 50)
    
    position_errors = []
    orientation_errors = []
    
    for i, (q_known, pos_known, ori_known) in enumerate(zip(known_joint_angles, known_position, known_orientation)):
        # Compute forward kinematics for known joint angles
        pos_computed = fk(q_known)  # Returns [x, y, z] in mm
        
        # Compute rotation matrix and extract orientation
        q_rad = np.radians(q_known)
        R_computed = np.array(rot_num(*q_rad), dtype=float)
        roll_comp, pitch_comp, yaw_comp = rotation_matrix_to_euler_zyx(R_computed)
        ori_computed = np.degrees([roll_comp, pitch_comp, yaw_comp])  # Convert to degrees
        
        # Calculate position error (mm)
        pos_error = np.linalg.norm(pos_computed - pos_known)
        position_errors.append(pos_error)
        
        # Calculate orientation error (degrees)
        def wrap_angle_deg(angle):
            return (angle + 180.0) % 360.0 - 180.0
        
        ori_error = np.array([
            wrap_angle_deg(ori_computed[0] - ori_known[0]),  # roll error
            wrap_angle_deg(ori_computed[1] - ori_known[1]),  # pitch error
            wrap_angle_deg(ori_computed[2] - ori_known[2])   # yaw error
        ])
        ori_error_norm = np.linalg.norm(ori_error)
        orientation_errors.append(ori_error_norm)
        
        # Print details for first few samples
        if i < 5:
            print(f"\nSample {i}:")
            print(f"  Joint angles: {q_known}")
            print(f"  Position (known):    [{pos_known[0]:7.2f}, {pos_known[1]:7.2f}, {pos_known[2]:7.2f}] mm")
            print(f"  Position (computed): [{pos_computed[0]:7.2f}, {pos_computed[1]:7.2f}, {pos_computed[2]:7.2f}] mm")
            print(f"  Position error: {pos_error:.4f} mm")
            print(f"  Orientation (known):    [{ori_known[0]:7.2f}, {ori_known[1]:7.2f}, {ori_known[2]:7.2f}] deg")
            print(f"  Orientation (computed): [{ori_computed[0]:7.2f}, {ori_computed[1]:7.2f}, {ori_computed[2]:7.2f}] deg")
            print(f"  Orientation error: {ori_error_norm:.4f} deg")
    
    # Summary statistics
    position_errors = np.array(position_errors)
    orientation_errors = np.array(orientation_errors)
    
    print(f"\n" + "=" * 50)
    print("FORWARD KINEMATICS ACCURACY SUMMARY")
    print("=" * 50)
    print(f"Number of test points: {len(position_errors)}")
    print(f"\nPosition Errors (mm):")
    print(f"  Mean: {np.mean(position_errors):.6f}")
    print(f"  Max:  {np.max(position_errors):.6f}")
    print(f"  Std:  {np.std(position_errors):.6f}")
    print(f"  RMS:  {np.sqrt(np.mean(position_errors**2)):.6f}")
    
    print(f"\nOrientation Errors (degrees):")
    print(f"  Mean: {np.mean(orientation_errors):.6f}")
    print(f"  Max:  {np.max(orientation_errors):.6f}")
    print(f"  Std:  {np.std(orientation_errors):.6f}")
    print(f"  RMS:  {np.sqrt(np.mean(orientation_errors**2)):.6f}")
    
    # Check if FK is accurate enough
    pos_threshold = 1.0  # 1mm threshold
    ori_threshold = 1.0  # 1 degree threshold
    
    pos_good = np.sum(position_errors < pos_threshold)
    ori_good = np.sum(orientation_errors < ori_threshold)
    
    print(f"\nAccuracy Assessment:")
    print(f"  Position errors < {pos_threshold} mm: {pos_good}/{len(position_errors)} ({100*pos_good/len(position_errors):.1f}%)")
    print(f"  Orientation errors < {ori_threshold}°: {ori_good}/{len(orientation_errors)} ({100*ori_good/len(orientation_errors):.1f}%)")
    
    if np.max(position_errors) > 10.0 or np.max(orientation_errors) > 10.0:
        print(f"\n⚠️  WARNING: Large FK errors detected!")
        print(f"   This could be the source of your inverse kinematics problems.")
    else:
        print(f"\n✅ Forward kinematics appears to be working correctly.")
        print(f"   Issues with inverse kinematics are likely in the optimization process.")
    
    return position_errors, orientation_errors

# Run the test 257.3736 mm 286.2646 mm
pos_errors, ori_errors = test_forward_kinematics()

Testing Forward Kinematics Accuracy

Sample 0:
  Joint angles: [ -0.17  23.37  16.17   4.3  -74.61  -2.9 ]
  Position (known):    [ 181.60,   -5.20,  204.50] mm
  Position (computed): [   9.00,    0.00,    0.01] mm
  Position error: 267.6415 mm
  Orientation (known):    [   1.64,   55.06,   -3.23] deg
  Orientation (computed): [ -90.00,   23.20,    0.00] deg
  Orientation error: 97.0741 deg

Sample 1:
  Joint angles: [-13.62  26.19   5.8   -3.42 -67.85  -2.98]
  Position (known):    [ 188.20,  -41.90,  215.50] mm
  Position (computed): [   8.86,    0.00,    1.18] mm
  Position error: 282.5824 mm
  Orientation (known):    [  -9.02,   53.72,  -20.86] deg
  Orientation (computed): [ -90.00,   12.57,    0.00] deg
  Orientation error: 93.1999 deg

Sample 2:
  Joint angles: [ 38.05  44.03 -13.71  28.65 -70.75 -19.07]
  Position (known):    [ 185.50,  107.80,  198.10] mm
  Position (computed): [   7.94,    0.00,   -3.08] mm
  Position error: 289.1771 mm
  Orientation (known):    [  15.66,   5

In [13]:
def _angle_diff_deg(a_deg, b_deg):
    """
    Compute signed angle difference in degrees, wrapped to [-180, 180].
    
    Args:
        a_deg, b_deg: Angles in degrees
        
    Returns:
        Signed difference a - b in degrees, mapped to [-180, 180]
    """
    d = (a_deg - b_deg + 180.0) % 360.0 - 180.0
    return d

results = []
max_nfev = 50000  # Increase from 5000
tol = 1e-9        # Tighten from 1e-6
reach = link_lengths(dh_table)
q_init = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

for i, (pos, ori, q_known) in enumerate(zip(known_position, known_orientation, known_joint_angles)):
    x, y, z = pos
    rx, ry, rz = ori
    try:
        q_sol = inverse_kinematics(x, y, z, rx, ry, rz, q_init, reach,
                                   max_iterations=max_nfev, tolerance=tol)
        q_sol = np.asarray(q_sol, dtype=float)  # in degrees
        per_joint_signed = np.array([_angle_diff_deg(q_sol[j], q_known[j]) for j in range(6)])
        per_joint_abs = np.abs(per_joint_signed)
        total_error = np.linalg.norm(per_joint_abs)
        status = "ok"
    except Exception as e:
        # mark failure and fill with NaNs
        q_sol = np.full(6, np.nan)
        per_joint_signed = np.full(6, np.nan)
        per_joint_abs = np.full(6, np.nan)
        total_error = np.nan
        status = f"fail: {e}"

    row = {
        "Index": i,
        "X": x, "Y": y, "Z": z,
        "RX": rx, "RY": ry, "RZ": rz,
        # known angles (degrees)
        "J1_known": q_known[0], "J2_known": q_known[1], "J3_known": q_known[2],
        "J4_known": q_known[3], "J5_known": q_known[4], "J6_known": q_known[5],
        # solved angles (degrees)
        "J1_sol": float(q_sol[0]) if not np.isnan(q_sol[0]) else np.nan,
        "J2_sol": float(q_sol[1]) if not np.isnan(q_sol[1]) else np.nan,
        "J3_sol": float(q_sol[2]) if not np.isnan(q_sol[2]) else np.nan,
        "J4_sol": float(q_sol[3]) if not np.isnan(q_sol[3]) else np.nan,
        "J5_sol": float(q_sol[4]) if not np.isnan(q_sol[4]) else np.nan,
        "J6_sol": float(q_sol[5]) if not np.isnan(q_sol[5]) else np.nan,
        # per-joint signed errors (degrees)
        "J1_err_signed": float(per_joint_signed[0]) if not np.isnan(per_joint_signed[0]) else np.nan,
        "J2_err_signed": float(per_joint_signed[1]) if not np.isnan(per_joint_signed[1]) else np.nan,
        "J3_err_signed": float(per_joint_signed[2]) if not np.isnan(per_joint_signed[2]) else np.nan,
        "J4_err_signed": float(per_joint_signed[3]) if not np.isnan(per_joint_signed[3]) else np.nan,
        "J5_err_signed": float(per_joint_signed[4]) if not np.isnan(per_joint_signed[4]) else np.nan,
        "J6_err_signed": float(per_joint_signed[5]) if not np.isnan(per_joint_signed[5]) else np.nan,
        # per-joint absolute errors (degrees)
        "J1_err_abs": float(per_joint_abs[0]) if not np.isnan(per_joint_abs[0]) else np.nan,
        "J2_err_abs": float(per_joint_abs[1]) if not np.isnan(per_joint_abs[1]) else np.nan,
        "J3_err_abs": float(per_joint_abs[2]) if not np.isnan(per_joint_abs[2]) else np.nan,
        "J4_err_abs": float(per_joint_abs[3]) if not np.isnan(per_joint_abs[3]) else np.nan,
        "J5_err_abs": float(per_joint_abs[4]) if not np.isnan(per_joint_abs[4]) else np.nan,
        "J6_err_abs": float(per_joint_abs[5]) if not np.isnan(per_joint_abs[5]) else np.nan,
        "Total_error_norm": float(total_error) if not np.isnan(total_error) else np.nan,
        "status": status
    }
    results.append(row)
    print(f"Row {i}: status={status}, total_error={row['Total_error_norm']:.6f}")

df_results = pd.DataFrame(results)
out_path = Path("/Users/yusufdineh/Documents/GitHub/EE347/Lab3") / "inverse_kinematics_results.csv"
df_results.to_csv(out_path, index=False)
print("Saved results to", out_path)
df_results

Row 0: status=fail: Target outside of range, total_error=nan
Row 1: status=fail: Target outside of range, total_error=nan
Row 2: status=fail: Target outside of range, total_error=nan
Row 3: status=fail: Target outside of range, total_error=nan
Row 4: status=fail: Target outside of range, total_error=nan
Row 5: status=fail: Target outside of range, total_error=nan
Row 6: status=fail: Target outside of range, total_error=nan
Row 7: status=fail: Target outside of range, total_error=nan
Row 8: status=fail: Target outside of range, total_error=nan
Row 9: status=fail: Target outside of range, total_error=nan
Row 10: status=fail: Target outside of range, total_error=nan
Row 11: status=fail: Target outside of range, total_error=nan
Row 12: status=fail: Target outside of range, total_error=nan
Row 13: status=fail: Target outside of range, total_error=nan
Saved results to /Users/yusufdineh/Documents/GitHub/EE347/Lab3/inverse_kinematics_results.csv


,Index,X,Y,Z,RX,RY,RZ,J1_known,J2_known,J3_known,...,J5_err_signed,J6_err_signed,J1_err_abs,J2_err_abs,J3_err_abs,J4_err_abs,J5_err_abs,J6_err_abs,Total_error_norm,status
0,0,181.6,-5.2,204.5,1.64,55.06,-3.23,-0.17,23.37,16.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail: Target outside of range
1,1,188.2,-41.9,215.5,-9.02,53.72,-20.86,-13.62,26.19,5.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail: Target outside of range
2,2,185.5,107.8,198.1,15.66,55.25,24.10,38.05,44.03,-13.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail: Target outside of range
3,3,168.6,4.7,249.1,85.23,-38.17,8.53,23.11,58.44,-84.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail: Target outside of range
4,4,212.0,10.3,198.3,86.67,-37.00,25.53,20.56,73.56,-84.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail: Target outside of range
5,5,187.8,18.8,174.7,97.47,-35.08,-1.23,24.34,77.16,-84.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail: Target outside of range
6,6,180.4,-4.7,220.4,-91.96,-23.52,-3.74,-21.62,65.65,-85.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail: Target outside of range
7,7,176.0,16.5,259.6,-75.76,-24.16,-11.20,-15.29,58.62,-85.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail: Target outside of range
8,8,213.0,6.7,210.0,-75.90,-29.20,-27.00,-15.46,73.38,-85.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail: Target outside of range
9,9,184.2,6.1,154.0,-89.52,-18.26,12.59,-16.08,84.99,-84.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail: Target outside of range
